<a href="https://colab.research.google.com/github/HuyenNguyenHelen/LING-5412/blob/main/Assignment_Feedforward_Network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import re
import shutil
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import losses
from tensorflow.keras import preprocessing
print(tf.__version__)


2.6.0


# Part 1: IMDB sentiment analysis

## Loading the dataset

In [2]:
url = 'https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz'
data = tf.keras.utils.get_file ('aclImdb_v1',
                                url,
                                untar = True,
                                cache_dir = '.',
                                cache_subdir = '')
data_dir = os.path.join (os.path.dirname(data), 'aclImdb')
print(os.listdir(data_dir))

84140032/84125825 [==============================] - 3s 0us/step
['imdb.vocab', 'test', 'README', 'imdbEr.txt', 'train']


In [3]:
train_dir = os.path.join (data_dir, 'train')
print(os.listdir(train_dir))

['neg', 'urls_neg.txt', 'urls_pos.txt', 'unsup', 'pos', 'urls_unsup.txt', 'unsupBow.feat', 'labeledBow.feat']


In [6]:
# Loading data from the directory
batch_size = 32
seed = 42
raw_train = tf.keras.utils.text_dataset_from_directory ('aclImdb/train',
                                                        batch_size =batch_size,
                                                        validation_split = 0.2,
                                                        subset = 'training',
                                                        seed = seed)
raw_val = tf.keras.utils.text_dataset_from_directory ('aclImdb/train',
                                                      batch_size = batch_size,
                                                      validation_split = 0.2,
                                                      subset = 'validation',
                                                      seed = seed)
raw_test = tf.keras.utils.text_dataset_from_directory ('aclImdb/test',
                                                       batch_size = batch_size)

Found 75000 files belonging to 3 classes.
Using 60000 files for training.
Found 75000 files belonging to 3 classes.
Using 15000 files for validation.
Found 25000 files belonging to 2 classes.


# Text representation

In [ ]:
def custom_preprocessing (text):
  lowercase = tf.strings.lower (text)
  stripped_html = tf.strings.regrex_replace (lowercase,'<br />', ' ')
  return tf/strings.regrex_replace(stripped_html,
                                   '')